# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

C:\ana3\envs\t_f2.2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ana3\envs\t_f2.2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    tmp = len(df[df[label] == 'yes'])/len(df)
    gini = 1 - (tmp**2+(1-tmp)**2)
    
    return gini    

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    tmp = df[attribute].unique()
    candidates=[]
    for i in range(len(tmp)-1):
        for k in combinations(tmp,i+1):
            candidates.append(list(k))
    result = candidates
    return result

In [6]:
get_binary_split(pd_data,"income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    
    gbs = get_binary_split(df,attribute)
    result = {}
    
    uni = df[attribute].unique()
    if len(uni) == 2:
        for i in gbs:
            tmp = '_'.join(i)
            yes = (len(df[df[attribute]==tmp])/len(df))*get_gini(df[df[attribute]==tmp],label)
            no = ((len(df)-len(df[df[attribute]==tmp]))/len(df))*get_gini(df[df[attribute]!=tmp],label)
            result[tmp] = yes+no
    else:
        a = uni[0]
        b = uni[1]
        c = uni[2]
        for i in gbs:
            tmp = '_'.join(i)
            if tmp == a + '_' + b:
                yes = (len(df[df[attribute].str.contains(a,b)])/len(df))*get_gini(df[df[attribute].str.contains(a,b)],label)
                no = ((len(df)-len(df[df[attribute].str.contains(a,b)]))/len(df))*get_gini(df[df[attribute].str.contains(c)],label)
                result[tmp] = yes+no
            elif tmp == a + '_' + c:
                yes = (len(df[df[attribute].str.contains(a,c)])/len(df))*get_gini(df[df[attribute].str.contains(a,c)],label)
                no = ((len(df)-len(df[df[attribute].str.contains(a,c)]))/len(df))*get_gini(df[df[attribute].str.contains(b)],label)
                result[tmp] = yes+no
            elif tmp == b + '_' + c:
                yes = (len(df[df[attribute].str.contains(b,c)])/len(df))*get_gini(df[df[attribute].str.contains(b,c)],label)
                no = ((len(df)-len(df[df[attribute].str.contains(b,c)]))/len(df))*get_gini(df[df[attribute].str.contains(a)],label)
                result[tmp] = yes+no
            else:
                yes = (len(df[df[attribute]==tmp])/len(df))*get_gini(df[df[attribute]==tmp],label)
                no = ((len(df)-len(df[df[attribute]==tmp]))/len(df))*get_gini(df[df[attribute]!=tmp],label)
                result[tmp] = yes+no
    return result

In [8]:
get_attribute_gini_index(pd_data, "income", "class_buys_computer")

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.4107142857142857,
 'high_low': 0.4603174603174603,
 'medium_low': 0.47619047619047616}

- 여기서 가장 작은 Gini index값을 가지는 class를 보여주세요.

In [9]:
tmp = get_attribute_gini_index(pd_data, "income", "class_buys_computer")
class_ = sorted(tmp)[0]
(class_,tmp[class_])

('high', 0.4428571428571429)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [10]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for column in features:
    
    kkk = get_attribute_gini_index(pd_data,column,'class_buys_computer')
    class_ = sorted(kkk)[0]
    print(f'{column}_minimum gini index:',(class_,kkk[class_]))

age_minimum gini index: ('middle_aged', 0.35714285714285715)
income_minimum gini index: ('high', 0.4428571428571429)
student_minimum gini index: ('no', 0.3673469387755103)
credit_rating_minimum gini index: ('excellent', 0.42857142857142855)


## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [11]:
# 각 변수를 대상으로 반복문 수행
df_tmp = pd_data[pd_data['age']=='middle_aged']
df_tmp_not = pd_data[pd_data['age']!='middle_aged']

for column in features[1:]:
    kkk = get_attribute_gini_index(df_tmp,column,'class_buys_computer')
    class_ = sorted(kkk)[0]
    print(f'Age : Middle_aged, Minimum Gini index of {column}:',(class_,kkk[class_]))

print("#############################################################")

"""
 두번째 데이터 프레임에 대한  Gini Index를 보여주세요.
 
"""

for column in features[1:]:
    kkk = get_attribute_gini_index(df_tmp_not,column,'class_buys_computer')
    class_ = sorted(kkk)[0]
    print(f'Age : Middle_aged, Minimum Gini index of {column}:',(class_,kkk[class_]))


Age : Middle_aged, Minimum Gini index of income: ('high', 0.0)
Age : Middle_aged, Minimum Gini index of student: ('no', 0.0)
Age : Middle_aged, Minimum Gini index of credit_rating: ('excellent', 0.0)
#############################################################
Age : Middle_aged, Minimum Gini index of income: ('high', 0.375)
Age : Middle_aged, Minimum Gini index of student: ('no', 0.31999999999999984)
Age : Middle_aged, Minimum Gini index of credit_rating: ('excellent', 0.4166666666666667)


## Entropy 를 구하는 함수 만들기

In [12]:
def getEntropy(df,feature):

    elements,counts = np.unique(df[feature],return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [13]:
getEntropy(pd_data,'class_buys_computer')

0.9402859586706311

In [14]:
def getGainA(df, feature) :
    info_D = getEntropy(df, feature) # 목표변수에 대한 Entropy 를 구하기
    columns = list(df.loc[:, df.columns != feature]) # 목표변수(feature)를 제외한 나머지 변수들의 변수명을 리스트 형태로 저장
    result = {}
    for col in columns:
        val,counts = np.unique(df[col],return_counts=True)
        weighted_entropy = np.sum([(counts[i]/np.sum(counts))*getEntropy(df.where(df[col]==val[i]).dropna(),feature) for i in range(len(val))])
        result[col] = info_D - weighted_entropy
    return result

In [15]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}